In [1]:
# !pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu

In [2]:
# !pip install llama-cpp-python

In [3]:
# !git lfs install
# !git clone https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF ../models/Llama-2-7B-Chat-GGUF

In [4]:
# !git clone --recursive -j8 https://github.com/abetlen/llama-cpp-python.git

In [5]:
# !set FORCE_CMAKE=1
# !set CMAKE_ARGS=-DLLAMA_CUBLAS=ON

In [6]:
# import langchain_community
# print(dir(langchain_community))

In [7]:
import os
import pandas as pd
from langchain_chroma import Chroma
from langchain.llms import LlamaCpp
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain.schema.runnable import RunnablePassthrough
from tqdm import tqdm

c:\Users\Thierry\miniconda3\envs\gutenberg\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

In [9]:
def load_csv(file_path):
    return pd.read_csv(file_path, sep=';')

In [10]:
class EmbeddingWrapper:
    def embed_documents(self, texts):
        embeddings = []
        for text in tqdm(texts, desc="Encoding documents"):
            embeddings.append(embeddings_model.encode(text))
        return embeddings

    def embed_query(self, query):
        return embeddings_model.encode([query])[0]

In [11]:
def create_vector_store(data):
    combined_texts = (data['Title'] + " " + data['Summary']).tolist()
    embedding_function = EmbeddingWrapper()
    vector_store = Chroma(
        collection_name="gutenberg_books",
        embedding_function=embedding_function,
        persist_directory="../models/chroma_langchain_db",
        )
    vector_store.add_texts(combined_texts)
    return vector_store

In [12]:
def configure_llama():
    model_path = os.path.abspath("../models/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q4_K_M.gguf")
    print(f"Loading Llama model from: {model_path}")
    try:
        llm = LlamaCpp(
            model_path=model_path,
            n_gpu_layers=40,
            n_batch=512,
            verbose=True,
        )
        print("Llama model loaded successfully.")
        return llm
    except Exception as e:
        print(f"Error loading Llama model: {e}")
        raise

In [13]:
# 5. Mise en place de la chaîne QA
def setup_qa_chain(llm, vector_store):
    template = """Contexte : Cet agent, basé sur un modèle de langage (LLM), 
    a pour objectif de répondre à des questions sur des livres provenant de Project Gutenberg. 
    Il est capable d'extraire des informations sur les livres et leurs personnages, 
    ainsi que d'interagir avec le texte complet d'un livre.
    Question: {question}
    Answer: Pour répondre à votre question, examinons d'abord les éléments clés et les détails pertinents. 
    Voici ce que nous savons :
    """
    prompt = PromptTemplate(template=template, input_variables=["question"])
    retriever = vector_store.as_retriever(k=5)
    qa_chain = RunnableSequence(
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
    )    
    return qa_chain

In [14]:
# Main
if __name__ == "__main__":
    file_path = os.path.join("../data", "gutenberg_cleaned.csv")
    data = load_csv(file_path)
    print(data.columns)  
    llm = configure_llama()
    vector_store = create_vector_store(data)
    qa_chain = setup_qa_chain(llm, vector_store)
    question = "Qui est l'auteur du livre Paradise Lost ?"
    response = qa_chain.invoke({"question": question})
    print(response)

Index(['Author', 'Title', 'Summary', 'Subject', 'EBook-No.', 'Release Date'], dtype='object')
Loading Llama model from: d:\FORMATIONS\SIMPLON\COURS\2024\S40\Brief_Gutenberg\models\Llama-2-7B-Chat-GGUF\llama-2-7b-chat.Q4_K_M.gguf


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


Llama model loaded successfully.


Encoding documents:   0%|          | 0/5000 [00:00<?, ?it/s]c:\Users\Thierry\miniconda3\envs\gutenberg\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Encoding documents: 100%|██████████| 5000/5000 [00:45<00:00, 109.56it/s]


 - L'auteur du livre "Paradise Lost" est John Milton.
     - Le livre a été publié en 1667.
     - Il s'agit d'une épopée religieuse qui raconte l'histoire de la chute d'Adam et Ève dans le jardin d'Éden.
    En utilisant ces informations, nous pouvons donc conclure que John Milton est l'auteur du livre "Paradise Lost".
    Nous espérons que cette réponse vous a été utile ! Si vous avez d'autres questions, n'hésitez pas à les poser.']
Réponse de l'agent :
L'auteur du livre "Paradise Lost" est John Milton.
Dans cette réponse, l'agent utilise des phrases telles que "examine d'abord les éléments clés et les détails pertinents" pour souligner l'importance de ces éléments dans la résolution de la question. L'agent utilise également des termes de base tels que "voici ce que nous savons" pour
